//Crawl data mobile

In [ ]:
pages = {
    'Apple':[
        'https://www.gsmarena.com/apple-phones-48.php',
        'https://www.gsmarena.com/apple-phones-f-48-0-p2.php',
        'https://www.gsmarena.com/apple-phones-f-48-0-p3.php',
    ],
    'Samsung':[
        'https://www.gsmarena.com/samsung-phones-9.php',
        'https://www.gsmarena.com/samsung-phones-f-9-0-p2.php',
        'https://www.gsmarena.com/samsung-phones-f-9-0-p3.php'
    ],
    'Huawei':[
        'https://www.gsmarena.com/huawei-phones-58.php',
        'https://www.gsmarena.com/huawei-phones-f-58-0-p2.php',
        'https://www.gsmarena.com/huawei-phones-f-58-0-p3.php'
    ],
    'LG':[
        'https://www.gsmarena.com/lg-phones-20.php',
        'https://www.gsmarena.com/lg-phones-f-20-0-p2.php',
        'https://www.gsmarena.com/lg-phones-f-20-0-p3.php'
    ],
    'Nokia':[
        'https://www.gsmarena.com/nokia-phones-1.php',
        'https://www.gsmarena.com/nokia-phones-f-1-0-p2.php',
        'https://www.gsmarena.com/nokia-phones-f-1-0-p3.php'
    ],
    'Xiaomi':[
        'https://www.gsmarena.com/xiaomi-phones-80.php',
        'https://www.gsmarena.com/xiaomi-phones-f-80-0-p2.php',
        'https://www.gsmarena.com/xiaomi-phones-f-80-0-p3.php'
    ],
    'Google':[
        'https://www.gsmarena.com/google-phones-107.php'
        ]
}
import csv
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor
import json
import csv
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
import tempfile
import time

# Read existing product names to skip those already processed
existing_products = set()
json_file = "output_data.json"
existing_data = []

# Load existing data from the JSON file if it exists
if os.path.exists(json_file):
    with open(json_file, mode='r', encoding='utf-8') as file:
        existing_data = json.load(file)
        for item in existing_data:
            product_name = item.get("PRODUCT_NAME")
            if product_name:
                existing_products.add(product_name)

def create_driver():
    options = Options()
    options.add_argument("--headless")  # Run in headless mode
    return webdriver.Chrome(options=options)

# Define a function to process each URL
def process_url(data, temp_file_path):
    branch, url = data
    driver = create_driver()
    wait = WebDriverWait(driver, 10)
    all_data = []  # Initialize an empty list to store all data objects

    try:
        driver.get(url)
        review_body = wait.until(EC.presence_of_element_located((By.ID, "review-body")))
        links = wait.until(lambda driver: review_body.find_elements(By.TAG_NAME, "a")) 
    except Exception as e:
        print(f"Error loading page {url}")
        driver.quit()
        return
    product_hrefs = []
    for link in links:
        # Chờ để tìm thấy phần tử có thẻ strong chứa tên sản phẩm
        product_name = wait.until(lambda driver: link.find_element(By.TAG_NAME, "strong")).text
        # Lấy href từ link
        href = link.get_attribute("href")
        # Thêm cặp product_name và href vào danh sách
        product_hrefs.append((product_name, href))
    for product_name, href in product_hrefs:
        try:
            if product_name in existing_products:
                continue
            print(href)
            driver.get(href)
            specs_list = wait.until(lambda driver: driver.find_element(By.ID, "specs-list"))
            tables = wait.until(lambda driver: specs_list.find_elements(By.TAG_NAME, "table"))
            data = {"PRODUCT_NAME": product_name, "BRANCH": branch}

            for table in tables:
                key = wait.until(lambda driver: table.find_element(By.TAG_NAME, "th"))
                tr_list = wait.until(lambda driver: table.find_elements(By.TAG_NAME, "tr"))
                master_key = key.text
                sub_data = {}
                previous_subkey = ''
                for tr in tr_list:
                    if tr.text != "":
                        td_list = wait.until(lambda driver: tr.find_elements(By.TAG_NAME, "td"))
                        subkey = td_list[0].text
                        subvalue = td_list[1].text
                        if subkey == '' or subkey == " ":
                            if master_key.lower() == 'network':
                                sub_data[previous_subkey] = sub_data[previous_subkey] + ";" + subvalue
                            else:
                                sub_data["OTHER"] = subvalue
                        else:
                            sub_data[subkey] = subvalue
                            previous_subkey = subkey
                        data[master_key] = sub_data

            all_data.append(data)  # Add data object to all_data list
            time.sleep(10)
            print(f"Success Link :{href}")
        except Exception as e:
            print(f"An error occurred for link {link}")
            continue

    # Save all_data to the temporary file
    with open(temp_file_path, "w", encoding="utf-8") as temp_file:
        json.dump(all_data, temp_file, ensure_ascii=False, indent=4)

    driver.quit()

# Flatten the URLs with their corresponding branches
urls_with_branches = [(branch, url) for branch, urls in pages.items() for url in urls]
# Create a list to store paths of temporary files for each thread
temp_files = []

# Run each (branch, url) tuple in a separate thread
with ThreadPoolExecutor(max_workers=3) as executor:  # Adjust max_workers as needed
    futures = []
    for data in urls_with_branches:
        # Create a temporary file for each thread
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".json", mode="w", encoding="utf-8")
        temp_files.append(temp_file.name)
        # Submit the process_url function with temp_file.name
        futures.append(executor.submit(process_url, data, temp_file.name))
    
    # Wait for all threads to complete
    for future in as_completed(futures):
        future.result()

# Combine existing data and data from all temporary JSON files into one

combined_data = existing_data
for temp_file_path in temp_files:
    with open(temp_file_path, "r", encoding="utf-8") as temp_file:
        temp_data = json.load(temp_file)
        combined_data.extend(temp_data)
    # Remove the temporary file after reading
    os.remove(temp_file_path)

# Write combined data to the final JSON file
with open("output_data.json", "w", encoding="utf-8") as json_file:
    json.dump(combined_data, json_file, ensure_ascii=False, indent=4)


Processing https://www.gsmarena.com/apple_iphone_5s-5685.php
Processing https://www.gsmarena.com/samsung_galaxy_s23-12082.php
Processing https://www.gsmarena.com/samsung_galaxy_f14_4g-13247.php
Processing https://www.gsmarena.com/apple_watch_series_7_aluminum-11107.php
Processing https://www.gsmarena.com/samsung_galaxy_z_flip3_5g-11044.php


Crawl CPU Data

In [30]:
import json
from bs4 import BeautifulSoup

# Load the HTML content from a file
with open('./Smartphone Processors - Benchmark List - NotebookCheck.net Tech.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Locate the table with the specific ID
table = soup.find('div', {'id':'c2052256'})

# Dictionary to store Model as key and Architecture as value
data_dict = {}

# Check if the table is found
if table:
    # Find the headers to locate the index of 'Model' and 'Architecture'
    trs = table.find_all('tr')
    
    for tr in trs:
        tds = tr.find_all('td')
        if tds[1].get_text() != "Model":
            pro = {tds[1].get_text() : tds[2].get_text()}
            data_dict[tds[1].get_text() ] =  tds[2].get_text()


# Save the data to a JSON file
with open('model_architecture.json', 'w', encoding='utf-8') as json_file:
    json.dump(data_dict, json_file, ensure_ascii=False, indent=4)

print("Data saved to model_architecture.json")


Data saved to model_architecture.json


In [38]:
import json
from bs4 import BeautifulSoup

# Load the HTML content from a file
with open('./iOS version history - Wikipedia.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Locate the table with the specific ID
table = soup.find('table', {'class':'wikitable'})

# Dictionary to store Model as key and Architecture as value
data_dict = {}

# Check if the table is found
if table:
    # Find the headers to locate the index of 'Model' and 'Architecture'
    trs = table.find_all('tr')
    
    for tr in trs:
        tds = tr.find_all('td')
        ths = tr.find_all('th')
        if len(tds) > 0 and len(ths) > 0:
            data_dict[ths[0].get_text().replace("\n","")] =  tds[0].get_text().replace("\n","")


# Save the data to a JSON file
with open('os_info.json', 'w', encoding='utf-8') as json_file:
    json.dump(data_dict, json_file, ensure_ascii=False, indent=4)

print("Data saved to model_architecture.json")

Data saved to model_architecture.json


In [7]:
import json
from bs4 import BeautifulSoup

def get_os_info(link, version_order, release_der, output, tableorder):
    # Load the HTML content from a file
    with open(link, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Locate the table with the specific ID
    table = soup.find_all('table', {'class':'wikitable'})[tableorder]

    # Dictionary to store Model as key and Architecture as value
    data_dict = {}

    # Check if the table is found
    if table:
        # Find the headers to locate the index of 'Model' and 'Architecture'
        trs = table.find_all('tr')
        
        for tr in trs:
            tds = tr.find_all('td')
            ths = tr.find_all('th')
            if len(tds) > 3:
                data_dict[("Android " + tds[version_order].get_text()).replace("\n","")] =  tds[release_der].get_text().replace("\n","")


    # Save the data to a JSON file
    with open(f"./os/{output}", 'w', encoding='utf-8') as json_file:
        json.dump(data_dict, json_file, ensure_ascii=False, indent=4)

    print("Data saved to model_architecture.json")

In [19]:
# HarmonyOS
get_os_info('./HarmonyOS version history - Wikipedia.html', 0, 3, 'os_info_har.json',1)

Data saved to model_architecture.json


STAGING DATA

In [1]:
import re
from datetime import datetime

def extract_and_format_date(text):
    # Sử dụng biểu thức chính quy để trích xuất đoạn có năm, tháng và ngày
    match = re.search(r'(\d{4}), (\w+) (\d{1,2})', text)
    if match:
        year = match.group(1)
        month_name = match.group(2)
        day = match.group(3)

        # Chuyển đổi tên tháng thành số
        date_str = f"{year} {month_name} {day}"
        date_obj = datetime.strptime(date_str, "%Y %B %d")

        # Định dạng lại ngày tháng
        formatted_date = date_obj.strftime("%Y/%m/%d")
        return formatted_date
    else:
        return "Không tìm thấy ngày tháng trong chuỗi."


def split_storage_and_ram(text):
    # Kiểm tra nếu text là None
    if text is None:
        return None, None

    # Sử dụng regex để tìm tất cả dung lượng lưu trữ và RAM
    storage = re.findall(r'\d+(?:GB|TB|MB)', text)
    ram = re.findall(r'\d+(?:GB|MB) RAM', text)
    
    # Tạo chuỗi từ danh sách dung lượng lưu trữ
    storage_str = ", ".join(storage) if storage else None
    
    # Kiểm tra các phần tử RAM có giống nhau không, nếu có phần tử
    if ram:
        # Xử lý chuỗi RAM bằng cách loại bỏ ' RAM' từ kết quả
        ram = [r.replace(' RAM', '') for r in ram]
        
        if all(ram[0] == r for r in ram):
            ram_str = ram[0]  # Lấy phần tử đầu tiên nếu tất cả giống nhau
        else:
            ram_str = ", ".join(ram)  # Lấy tất cả nếu không giống nhau
    else:
        ram_str = None  # Nếu không có RAM thì trả về None

    return storage_str, ram_str

def remove_parentheses(text):
    # Remove text inside parentheses, including the parentheses
    cleaned_text = re.sub(r'\(.*?\)', '', text)

    return cleaned_text.strip()
def get_os(text):
    # Tách chuỗi bằng dấu phẩy
    parts = text.split(", ")
    return parts[0]



In [2]:
import json

# Load the JSON files
with open('./output_data.json', 'r') as file:
    data = json.load(file)
with open('./model_architecture.json', 'r') as file:
    arc_data = json.load(file)
with open('./os/os_info.json', 'r') as file:
    os_data = json.load(file)
with open('./company_info.json', 'r') as file:
    company_data = json.load(file)

stg_data = []
for item in data:
    if "Watch" in item.get('PRODUCT_NAME', ""):
        continue
    storage_str, ram_str = split_storage_and_ram(item.get('MEMORY', {}).get('Internal', None))
    data_dict = {
        'name': item.get('PRODUCT_NAME', None),
        'releaseDate': extract_and_format_date(item.get('LAUNCH', {}).get('Status', None)),
        'simType': item.get('NETWORK', {}).get('Technology', None),
        'storageCapacity': storage_str,
        'ramSize': ram_str,
        'Screen': {
            'type': item.get('DISPLAY', {}).get('Type', None),
            'size': item.get('DISPLAY', {}).get('Size', None),
            'resolution': item.get('DISPLAY', {}).get('Resolution', None)
        },
        'CPU': {
            'name': item.get('PLATFORM', {}).get('Chipset', None),
            'parameter': item.get('PLATFORM', {}).get('CPU', None),
            'arch': arc_data.get(remove_parentheses(item.get('PLATFORM', {}).get('Chipset', '')), None)
        },
        'Battery': {
            'capacity': item.get('BATTERY', {}).get('Type', '').split(',')[0] if item.get('BATTERY', {}).get('Type') else None,
            'charge': item.get('BATTERY', {}).get('Charging', None)
        },
        'OS': {
            'name': item.get('PLATFORM', {}).get('OS', None),
            'releaseDate': os_data.get(remove_parentheses(get_os(item.get('PLATFORM', {}).get('OS', ''))), None)
        },
        'FrontCamera': {
            'type': next(iter(item.get("SELFIE CAMERA", {})), None),
            'resolution': item.get("SELFIE CAMERA", {}).get(next(iter(item.get("SELFIE CAMERA", {})), ''), None),
            'video': item.get("SELFIE CAMERA", {}).get('Video', None),
            'features': item.get("SELFIE CAMERA", {}).get('Features', None)
        },
        'MainCamera': {
            'type': next(iter(item.get("MAIN CAMERA", {})), None),
            'resolution': item.get("MAIN CAMERA", {}).get(next(iter(item.get("MAIN CAMERA", {})), ''), None),
            'video': item.get("MAIN CAMERA", {}).get('Video', None),
            'features': item.get("MAIN CAMERA", {}).get('Features', None)
        },
        'Brand': {
            'name': item.get('BRANCH', None),
            'country': company_data.get(item.get('BRANCH', ''), {}).get("country", None),
            'foundDate': company_data.get(item.get('BRANCH', ''), {}).get("foundDate", None)
        }
    }
    stg_data.append(data_dict)
with open(f"./data_ontology_stg.json", 'w', encoding='utf-8') as json_file:
    json.dump(stg_data, json_file, ensure_ascii=False, indent=4)


In [1]:
import json
import rdflib
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import XSD
import re
from datetime import datetime

# Define the namespace for your ontology
SMP = Namespace("http://www.semanticweb.org/hust/master/2024/10/ontologies/smartphone.owl#")

# Function to clean strings for URI creation
def clean_string_for_uri(value):
    return re.sub(r'[^\w]', '_', value).lower()

# Function to convert date formats to YYYY-MM-DD
def format_date(date_str):
    try:
        if '/' in date_str:
            return datetime.strptime(date_str, "%Y/%m/%d").strftime("%Y-%m-%d")
        else:
            return datetime.strptime(date_str, "%B %d, %Y").strftime("%Y-%m-%d")
    except:
        return None

# Load the JSON data from a file
with open('data_ontology_stg.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# Create the graph
g = Graph()
g.bind("smp", SMP)

# Cache for shared URIs to reuse
uri_cache = {}

def get_or_create_shared_uri(prop_type, value):
    if not value:  # Check if value is None or empty
        return None
    clean_value = clean_string_for_uri(value)
    uri_key = f"{prop_type}_{clean_value}"
    
    if uri_key not in uri_cache:
        uri_cache[uri_key] = SMP[clean_value]
    
    return uri_cache[uri_key]

# Iterate over each smartphone object in the list
for smartphone in json_data:
    product_name_clean = clean_string_for_uri(smartphone["name"])
    product_uri = SMP[product_name_clean]

    # Add main smartphone attributes
    g.add((product_uri, RDF.type, SMP.SmartPhone))
    g.add((product_uri, SMP.name, Literal(smartphone["name"], datatype=XSD.string)))

    # Handle release date
    if smartphone["releaseDate"]:
        release_date = format_date(smartphone["releaseDate"])
        if release_date:
            g.add((product_uri, SMP.releaseDate, Literal(release_date, datatype=XSD.date)))

    # Add simType and storageCapacity
    g.add((product_uri, SMP.simType, Literal(smartphone["simType"], datatype=XSD.string)))
    g.add((product_uri, SMP.storageCapacity, Literal(smartphone["storageCapacity"], datatype=XSD.string)))
    g.add((product_uri, SMP.ramSize, Literal(smartphone["ramSize"], datatype=XSD.string)))

    # Handle Screen properties
    if "Screen" in smartphone:
        screen_uri = SMP[product_name_clean + "_screen"]
        g.add((screen_uri, RDF.type, SMP.Screen))
        g.add((screen_uri, SMP.type, Literal(smartphone["Screen"]["type"], datatype=XSD.string)))
        g.add((screen_uri, SMP.size, Literal(smartphone["Screen"]["size"], datatype=XSD.string)))
        g.add((screen_uri, SMP.resolution, Literal(smartphone["Screen"]["resolution"], datatype=XSD.string)))
        g.add((product_uri, SMP.hasScreen, screen_uri))

    # Handle CPU properties
    if "CPU" in smartphone and smartphone["CPU"]["name"]:
        cpu_name = smartphone["CPU"]["name"]
        cpu_uri = get_or_create_shared_uri("cpu", cpu_name)
        if cpu_uri:  # Check if uri is not None
            g.add((cpu_uri, RDF.type, SMP.CPU))
            g.add((cpu_uri, SMP.name, Literal(cpu_name, datatype=XSD.string)))
            g.add((cpu_uri, SMP.parameter, Literal(smartphone["CPU"]["parameter"], datatype=XSD.string)))
            g.add((cpu_uri, SMP.arch, Literal(smartphone["CPU"]["arch"], datatype=XSD.string)))
            g.add((product_uri, SMP.hasCPU, cpu_uri))

    # Handle Battery properties (which may be unique per product)
    if "Battery" in smartphone:
        battery_uri = SMP[product_name_clean + "_battery"]
        g.add((battery_uri, RDF.type, SMP.Battery))
        g.add((battery_uri, SMP.capacity, Literal(smartphone["Battery"]["capacity"], datatype=XSD.string)))
        g.add((product_uri, SMP.hasBattery, battery_uri))

    # Handle OS properties
    if "OS" in smartphone and smartphone["OS"]["name"]:
        os_name = smartphone["OS"]["name"]
        os_uri = get_or_create_shared_uri("os", os_name)
        if os_uri:  # Check if uri is not None
            g.add((os_uri, RDF.type, SMP.OS))
            g.add((os_uri, SMP.name, Literal(os_name, datatype=XSD.string)))
            if smartphone["OS"]["releaseDate"]:
                os_release_date = format_date(smartphone["OS"]["releaseDate"])
                if os_release_date:
                    g.add((os_uri, SMP.releaseDate, Literal(os_release_date, datatype=XSD.date)))
            g.add((product_uri, SMP.hasOS, os_uri))

    # Handle Camera properties (Front and Main)
    for camera_type in ["FrontCamera", "MainCamera"]:
        if camera_type in smartphone:
            camera_uri = SMP[product_name_clean + "_" + camera_type.lower()]
            camera_class = SMP.FrontCamera if camera_type == "FrontCamera" else SMP.MainCamera
            g.add((camera_uri, RDF.type, camera_class))
            g.add((camera_uri, SMP.type, Literal(smartphone[camera_type]["type"], datatype=XSD.string)))
            g.add((camera_uri, SMP.resolution, Literal(smartphone[camera_type]["resolution"], datatype=XSD.string)))
            g.add((camera_uri, SMP.video, Literal(smartphone[camera_type]["video"], datatype=XSD.string)))
            g.add((camera_uri, SMP.features, Literal(smartphone[camera_type]["features"], datatype=XSD.string)))
            g.add((product_uri, SMP.hasCamera, camera_uri))

    # Handle Brand properties
    if "Brand" in smartphone and smartphone["Brand"]["name"]:
        brand_name = smartphone["Brand"]["name"]
        brand_uri = get_or_create_shared_uri("brand", brand_name)
        if brand_uri:  # Check if uri is not None
            g.add((brand_uri, RDF.type, SMP.Brand))
            g.add((brand_uri, SMP.name, Literal(brand_name, datatype=XSD.string)))
            g.add((brand_uri, SMP.country, Literal(smartphone["Brand"]["country"], datatype=XSD.string)))
            if smartphone["Brand"]["foundDate"]:
                brand_found_date = format_date(smartphone["Brand"]["foundDate"])
                if brand_found_date:
                    g.add((brand_uri, SMP.foundDate, Literal(brand_found_date, datatype=XSD.date)))
            g.add((product_uri, SMP.hasBrand, brand_uri))

    if "RAM" in smartphone and smartphone["OS"]["name"]:
        os_name = smartphone["OS"]["name"]
        os_uri = get_or_create_shared_uri("os", os_name)
        if os_uri:  # Check if uri is not None
            g.add((os_uri, RDF.type, SMP.OS))
            g.add((os_uri, SMP.name, Literal(os_name, datatype=XSD.string)))
            if smartphone["OS"]["releaseDate"]:
                os_release_date = format_date(smartphone["OS"]["releaseDate"])
                if os_release_date:
                    g.add((os_uri, SMP.releaseDate, Literal(os_release_date, datatype=XSD.date)))
            g.add((product_uri, SMP.hasOS, os_uri))
# Serialize to RDF/XML format
rdf_data = g.serialize(format='xml')

# Save the RDF data to a file
with open("smartphone.rdf", "w") as f:
    f.write(rdf_data)

print("RDF data has been successfully generated and saved.")


RDF data has been successfully generated and saved.


In [6]:
import xml.etree.ElementTree as ET
import csv

# Hàm để kiểm tra và thêm dấu ngoặc kép nếu chưa có
def add_quotes_if_needed(value):
    if value and not (value.startswith('"') and value.endswith('"')):
        return f'"{value}"'
    return value

# Hàm để chuyển RDF XML thành CSV
def rdf_to_csv(rdf_file, csv_file):
    # Parse XML file
    tree = ET.parse(rdf_file)
    root = tree.getroot()

    # Danh sách các smartphone (mỗi smartphone là một dòng trong CSV)
    data = []

    # Duyệt qua từng thẻ SmartPhone
    for smartphone in root.findall('{http://www.semanticweb.org/hust/master/2024/10/ontologies/smartphone.owl/}SmartPhone'):
        row = {}
        # Lấy URI từ thuộc tính rdf:about
        row['uri'] = smartphone.attrib.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about', '')

        # Duyệt qua các thuộc tính con của smartphone
        for child in smartphone:
            # Lấy tên thẻ và giá trị
            tag = child.tag.split('}')[-1]  # Lấy phần sau dấu ngoặc nhọn nếu có namespace

            # Nếu thẻ con có thêm các thẻ con khác (ví dụ: Screen, CPU)
            if len(child):
                for sub_child in child:
                    sub_tag = sub_child.tag.split('}')[-1]
                    row[sub_tag] = sub_child.text
            else:
                row[tag] = child.text

        # Thêm dòng dữ liệu vào danh sách
        data.append(row)

    # Lấy các tên cột mà không thêm cột 'uri' lần thứ hai
    columns = ['uri'] + [key for key in data[0] if key != 'uri']

    # Ghi ra file CSV
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_MINIMAL)
        
        # Ghi tiêu đề cột
        writer.writerow(columns)

        # Ghi từng dòng dữ liệu, bọc giá trị trong dấu ngoặc kép nếu cần
        for row in data:
            writer.writerow([add_quotes_if_needed(row.get(col, '')) for col in columns])

# Gọi hàm để chuyển đổi file RDF thành CSV
rdf_to_csv('output.rdf', 'output.csv')


In [8]:
import rdflib
import csv

# Define the namespace for your ontology
SMP = rdflib.Namespace("http://www.semanticweb.org/hust/master/2024/10/ontologies/smartphone.owl#")

# Load the RDF/XML file
rdf_file = 'smartphone.rdf'
g = rdflib.Graph()
g.parse(rdf_file, format='xml')

# List to store rows (uri and product_name)
rows = []

# Query for RDF descriptions of SmartPhone
for subject, predicate, obj in g.triples((None, rdflib.RDF.type, SMP.SmartPhone)):
    # Extract the URI (subject) and product name
    product_uri = subject
    product_name = None

    # Look for the product name in the triples
    for _, p, o in g.triples((subject, SMP.name, None)):
        product_name = str(o)

    # Append the URI and product name to the list
    if product_name:
        rows.append([product_uri, product_name])

# Write to CSV
csv_file = 'smartphone.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['uri', 'product_name'])  # Write header
    writer.writerows(rows)  # Write data

print(f"CSV file '{csv_file}' has been created successfully.")


CSV file 'smartphone.csv' has been created successfully.


In [3]:
from rdflib import Graph

# Load the RDF/XML file
g = Graph()
try:
    g.parse("D:/study/SematicWeb/crawl_mobile/smartphone.owl.xml", format="xml")
    print("RDF/XML syntax is valid.")
except Exception as e:
    print(f"Error in RDF/XML file: {e}")


RDF/XML syntax is valid.


In [12]:
from rdflib import Graph, Namespace

# Load ontology và RDF/XML
ontology_file = './smartphone.owl.xml'
data_file = './output_v2.rdf'

g_ontology = Graph()
g_data = Graph()

# Parse các file
g_ontology.parse(ontology_file)
g_data.parse(data_file)

# Namespace của ontology
n = Namespace('http://www.semanticweb.org/hust/master/2024/10/ontologies/smartphone.owl/')

# Kiểm tra xem các thuộc tính trong RDF có xuất hiện trong ontology hay không
for s, p, o in g_data:
    if p not in g_ontology:
        print(f"Thuộc tính {p} không có trong ontology")
    else:
        print(f"Thuộc tính {p} hợp lệ")

print("Kiểm tra hoàn tất!")


ValueError: too many values to unpack (expected 3)